<h2>주가 상승/하락 일자별 뉴스 단어 빈도

# install package

In [4]:
pip install selenium

In [5]:
pip install konlpy

# drive mount

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# import

In [7]:
import time, re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm
from datetime import datetime, timedelta

from selenium.webdriver.common.keys import Keys

from konlpy.tag import *

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, Dense, LSTM, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import warnings
warnings.filterwarnings('ignore')

In [8]:
pd.options.display.float_format = '{:.5f}'.format

# 전처리

## clean_text

In [9]:
def clean_text(inputString):
    text_rmv = re.sub('[+]','증가', inputString)
    text_rmv = re.sub('[-]','감소', text_rmv)
    text_rmv = re.sub('[↑▲]','상승', text_rmv)
    text_rmv = re.sub('[↓△]','하락', text_rmv)
    text_rmv = re.sub('[^一-龥朴文ㄱ-힣_]', ' ', text_rmv)


    return text_rmv

## clean_text_2

In [10]:
def clean_text_2(text):
    patterns = [
        r'\.([^\.]*(?:기자|특파원|교수|작가|대표|논설|고문|주필|부문장|팀장|장관|원장|연구원|이사장|위원|실장|차장|부장|에세이|화백|사설|소장|단장|과장|기획자|큐레이터|저작권|평론가|©|©|ⓒ|\@|\/|=|▶|무단|전재|재배포|금지|\[|\]|\(\))[^\.]*)$',
        r'원본보기',
        r'아이콘',
        r'AD',
        r'자료',
        r'ADVERTISEMENT',
        r'Advertisement',
        r'viewer'
    ]

    for pattern in patterns:
        regex = re.compile(pattern)
        text = re.sub(regex, '', text)

    text = re.sub('[^\w.,%]', '', text)
    text = ' '.join(text.split())
    text = re.sub(r"\S+@\S+", "", text)  # 이메일 패턴 제거
    text = re.sub(r"[가-힣]+기자", "", text)  # 기자 이름 패턴 제거 (한글 기자 이름)
    text = re.sub(r'\b\w+\s?기자\b', '', text)
    text = re.sub(r'\b\w+@\w+\.\w+\b', '', text)
    text = re.sub(r'\b\w+\.com\b', '', text)
    # text = re.sub(r'[^.!?]+$', '', text)
    text = re.sub(r'[.!?]\s*$', '', text)


    return text

## 데이터 프레임 생성

In [11]:
df = pd.read_excel("/content/drive/MyDrive/유비온 프로젝트1/datasets/LG화학 전처리(완).xlsx", index_col=0)

## 본문 전처리 적용

In [12]:
# df["article_text"] = df["article_text"].apply(clean_text)

In [13]:
# df["article_text"] = df["article_text"].apply(clean_text_2)

In [14]:
# Title 전처리 및 df['noun'] 생성 -> Title 에서 명사만 생성
n_ = []
okt = Okt()

In [15]:
# 2022년 1월 1일,2일 데이터 삭제로 인해 인덱스 안맞음 ; 인덱스 초기화
df.reset_index(inplace=True)

In [ ]:
for i in tqdm(range(len(df))) :
    # 바뀌어진 기사 제목에서 okt로 명사만 뽑기
    n_.append(' '.join(okt.nouns(df.loc[i,'article_text'])))

df['nouns'] = n_

 98%|█████████▊| 475/484 [09:06<00:08,  1.09it/s]

# 일자별 뉴스 각 단어에 점수 매기기

In [ ]:
# nan확인용
df.loc[df["nouns"].isna()].index

In [ ]:
# 뉴스 전체에 있는 단어 파악 & 카운팅
vocab = {}

# LSTM 돌릴 때 각 원소 길이 맞추기 위해 등장하지 않은 단어에 대해
# 0점을 부여하기 위해 만들어놓은 임시 딕셔너리
# 밑에 코드에서 다시 쓰임
vocab_freq_init = {}

# 일자별 뉴스에 있는 단어 파악 & 카운팅한 정보를 각 일자별로 모두 담는 배열
vocab_date_list = []

for i in tqdm(range(len(df))):
    nouns = df.loc[i,"nouns"]
    nouns = nouns.split(' ')
    # 일자별 뉴스에 있는 단어 파악 & 카운팅하기 위한 딕셔너리
    vocab_date = {}

    for n in nouns :
        # 띄어쓰기 단위로 끊어진 각 단어 중 한 글자 짜리는 카운트 하지 않음
        if len(n) > 1:
            if n in vocab:
                t = vocab[n]
                vocab[n] = t + 1

                if n in vocab_date:
                    t_ = vocab_date[n]
                    vocab_date[n] = t_ + 1
                else:
                    vocab_date[n] = 1
            else:
                vocab[n] = 1
                # vocab_date[n] = 1
            vocab_freq_init[n] = 0
        else:
            pass
    print(f"{i} -  {sorted(vocab_date.items(), key=lambda x:x[1], reverse=True)}\n")
    vocab_date_list.append(vocab_date)

 27%|██▋       | 133/484 [00:00<00:00, 1326.64it/s]

0 -  [('전일', 39), ('코스피', 26), ('대비', 20), ('증권', 19), ('정보', 19), ('현재', 19), ('거래', 19), ('상승', 12), ('종목', 6), ('출발', 6), ('외국인', 5), ('전자', 5), ('하락', 5), ('기관', 5), ('삼성', 5), ('카카오', 5), ('마감', 4), ('네오위즈', 3), ('페이', 3), ('코스닥', 3), ('전기', 3), ('첨단', 3), ('현대차', 3), ('성전', 2), ('가장', 2), ('화학', 2), ('최고', 2), ('빅스', 2), ('나스닥', 2), ('반등', 2), ('예상', 2), ('엔지니어링', 2), ('글로벌', 2), ('위메이드', 2), ('프로', 2), ('에너지', 2), ('클릭', 2), ('바이오', 2), ('게임', 2), ('효성', 2), ('소재', 2), ('기아', 2), ('디스플레이', 1), ('학회', 1), ('텔레콤', 1), ('가족', 1), ('로밍', 1), ('하이닉스', 1), ('기업', 1), ('기본', 1), ('인상', 1), ('물산', 1), ('투자', 1), ('남국', 1), ('사태', 1), ('에코', 1), ('전환', 1), ('이노베이션', 1), ('공유', 1), ('배터리', 1), ('감소', 1), ('직스', 1), ('신청', 1), ('팔자', 1), ('수익', 1), ('전략', 1), ('시장', 1)]

1 -  [('감소', 4), ('전자', 3), ('시장', 2), ('종목', 2), ('삼성', 2), ('카카오', 2), ('현대차', 2), ('상승', 1), ('황소', 1), ('사진', 1), ('증권', 1), ('매수', 1), ('주가', 1), ('코스피', 1), ('성전', 1), ('확인', 1), ('거래', 1), ('모비스', 1), ('가장', 1), ('손

 55%|█████▍    | 266/484 [00:00<00:00, 1027.97it/s]

In [ ]:
print("전체뉴스 단어&카운트")
print(vocab)
print()
print("일자별 단어&카운트 배열")
for i, vocab_date in enumerate(vocab_date_list):
    print(f"{i} - {vocab_date}")

In [ ]:
t_under_10 = []

for k, v in vocab.items():
    if vocab[k]<10:
        t_under_10.append(k)

for k in t_under_10:
    del vocab[k]

In [ ]:
for k,v in vocab.items():
    df[k]=0

In [ ]:
for j in range(len(vocab_date_list)):
    vocab_dict = vocab_date_list[j]
    for i,(key,value) in enumerate(vocab_dict.items()):
        if key in t_under_10:
            continue
        else:
            vocab_freq = vocab_dict[key] / vocab[key] * 100
        df[key][j] = vocab_freq

NameError: ignored

In [ ]:
df.to_excel("/content/drive/MyDrive/유비온 프로젝트1/datasets/freq_df_LG.xlsx")

## 유가데이터

In [ ]:
df = pd.read_excel("/content/drive/MyDrive/유비온 프로젝트1/datasets/LG화학 전처리(완).xlsx", index_col=0)

In [ ]:
oil_ex = pd.read_excel("/content/drive/MyDrive/유비온 프로젝트1/datasets/dh(전처리찐막).xlsx")
oil_article_preprocessed = oil_ex["oil_news"]

In [ ]:
df["oil_news"] = oil_article_preprocessed

In [ ]:
# Title 전처리 및 df['noun'] 생성 -> Title 에서 명사만 생성
n_ = []
okt = Okt()

for i in tqdm(range(len(df))):
    # 바뀌어진 기사 제목에서 okt로 명사만 뽑기
    n_.append(' '.join(okt.nouns(df.loc[i,'oil_news'])))

df['nouns_oil'] = n_

In [ ]:
vocab_oil = {}
vocab_freq_init_oil = {}
vocab_date_list_oil = []

for i in tqdm(range(len(df))):
    nouns = df.loc[i,"nouns_oil"]
    nouns = nouns.split(' ')
    vocab_date = {}

    for n in nouns :
        # 띄어쓰기 단위로 끊어진 각 단어 중 한 글자 짜리는 카운트 하지 않음
        if len(n) > 1:
            if n in vocab_oil:
                t = vocab_oil[n]
                vocab_oil[n] = t + 1

                if n in vocab_date:
                    t_ = vocab_date[n]
                    vocab_date[n] = t_ + 1
                else:
                    vocab_date[n] = 1
            else:
                vocab_oil[n] = 1
                # vocab_date[n] = 1
            vocab_freq_init_oil[n] = 0
        else:
            pass
    print(f"{i} -  {vocab_date}")
    vocab_date_list_oil.append(vocab_date)

 82%|████████▏ | 396/483 [00:00<00:00, 1978.98it/s]

0 -  {'계속': 1, '대비': 6, '상승': 7, '공업': 1, '제품': 1, '서비스': 1, '신선': 1, '동월': 3, '물가': 1, '지수': 1, '전월': 2, '기록': 2, '물가상승률': 1, '충북': 1, '충남': 1, '대전': 1, '세종': 1, '국제': 2, '가격': 2, '전망': 1, '에너지': 2, '수요': 1}
1 -  {'에너지': 1, '코로나': 2, '바이러스': 1, '오미크론': 1, '글로벌': 1, '유가': 4, '배럴': 3, '최근': 1, '내년': 1, '국제': 2, '가격': 9, '제품': 4, '플라스틱': 11, '확대': 1, '새해': 1, '전망': 1, '활용': 2, '소재': 4, '회복': 1, '석유화학': 2, '급등': 2, '에틸렌': 1, '위해': 2, '재활용': 4, '시장': 1, '재생': 2, '격차': 1, '감소': 1, '때문': 1, '업계': 1, '조달': 1, '다변': 1, '화해': 1, '변동': 1, '극복': 1, '달성': 1, '품질': 2, '확보': 1, '그린': 1, '식품': 1, '용기': 4, '친환경': 1, '지역': 1, '케미칼': 1, '협약': 1, '수거': 2, '페트병': 1, '수요': 1, '증가': 1, '이상승': 1, '스타트업': 1, '사례': 1, '배달': 4, '원료': 1, '상승': 1}
2 -  {'저작권': 2, '무단': 2, '재및재': 2, '배포': 2, '금지': 2, '지수': 2, '상승': 1, '포인트': 3, '하락': 2, '오미크론': 1, '금리': 1, '급등': 1, '나스닥': 1, '미국': 2, '코로나': 1, '실제': 1, '회복': 1, '관련': 1, '종목': 1, '계속': 1, '투자': 2, '영향': 3, '기록': 2, '전날': 1, '국제': 2, '유가': 3, '뉴욕': 1, '거래소': 1, '가격'

100%|██████████| 483/483 [00:00<00:00, 1870.33it/s]

427 -  {'울산': 2, '지역': 2, '제조업': 4, '업황': 2, '올해': 2, '한국은행': 1, '전월': 22, '무려': 1, '하락': 3, '작년': 1, '이후': 2, '가장': 1, '석유화학': 4, '금속': 1, '제품': 3, '업종': 1, '중심': 2, '원자재': 3, '가격': 5, '부담': 1, '지수': 2, '악화': 1, '때문': 1, '분석': 1, '준치': 3, '긍정': 2, '응답': 3, '업체': 3, '정적': 1, '수가': 1, '상이': 1, '상황': 5, '기업': 3, '미만': 1, '반대': 1, '의미': 1, '매출': 3, '생산': 4, '신규': 2, '수주': 2, '재고': 1, '수준': 2, '가동': 2, '설비': 3, '투자': 1, '산성': 3, '구입': 1, '판매': 1, '자금': 4, '사정': 4, '인력': 2, '분류': 1, '실적': 9, '감소': 1, '상승': 14, '대비': 12, '비교': 1, '다음': 2, '전망': 3, '가운데': 2, '과인': 1, '인건비': 2, '불확실': 2, '한경제': 2, '애로': 2, '사항': 2, '꼽았': 1, '자재': 1, '수출': 1, '부진': 3, '비중': 2, '부족': 1, '비제': 1, '조업': 1, '경우': 1, '개선': 2, '소매업': 1, '건설': 1, '폐기물': 1, '처리': 1, '등기': 1, '영향': 2, '도전': 1, '급증': 1, '앞서': 1, '상공회의소': 1, '최근': 2, '제조업체': 1, '대상': 1, '실시': 1, '자동차': 1, '부품': 1, '회복': 1, '한기': 1, '대감': 1, '경기': 1, '집계': 1, '상회': 1, '정유': 3, '직전': 1, '분기': 1, '반등': 1, '내외': 1, '경제지표': 1, '정상화': 1, '기전': 1, '낙관': 1, '지적':

In [ ]:
t_under_10_oil = []
for k, v in vocab_oil.items():
    if vocab_oil[k]<10:
        t_under_10_oil.append(k)

for k in t_under_10_oil:
    del vocab_oil[k]

In [ ]:
for k,v in vocab_oil.items():
    df[k]=0

<ipython-input-84-0bf1f3e40d6d>:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df2[k]=0
<ipython-input-84-0bf1f3e40d6d>:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df2[k]=0
<ipython-input-84-0bf1f3e40d6d>:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df2[k]=0
<ipython-input-84-0bf1f3e40d6d>

In [ ]:
for j in range(len(vocab_date_list_oil)):
    vocab_dict = vocab_date_list_oil[j]
    for i,(key,value) in enumerate(vocab_dict.items()):
        if key in t_under_10_oil:
            continue
        else:
            vocab_freq = vocab_dict[key] / vocab_oil[key] * 100
        df[key][j] = vocab_freq

<ipython-input-85-777afaf5a306>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[key][j] = vocab_freq


In [ ]:
df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'index', 'predict_date', 'oil_news',
       'article_text', '대한종가', '대한타겟', 'nouns', 'nouns_freq',
       ...
       '이륜차', '파산', '경제의회', '우즈', '가액', '취업', '실리콘밸리', '펑크', '디폴트', '매카시'],
      dtype='object', length=2167)

In [ ]:
df["디폴트"][180]

0.0

In [ ]:
df.to_excel("/content/drive/MyDrive/유비온 프로젝트1/datasets/oil_freq_df.xlsx")

In [ ]:
# for j in range(len(vocab_date_list_oil)):
#     vocab_dict = vocab_date_list_oil[j]
#     vocab_freq = vocab_freq_init_oil
#     for i,(key,value) in enumerate(vocab_dict.items()):
#         if key in t_under_10_oil:
#             pass
#         else:
#             vocab_freq[key] = vocab_dict[key] / vocab_oil[key] * 100

#     df["nouns_freq_oil"][j] = vocab_freq.values()

In [ ]:
# df.to_excel("./word_freq_embedding_final.xlsx")

<h1> ______________________전처리 끝________________________